生成器generator是从其他语言中弄出来的新语法,和python一样,生成器的作用也是构造无限流和实现异步协程,异步部分会在后面细讲

生成器的定义方式和function差不多,只是关键字function后面添个`*`号,return 改成 yield

In [1]:
import 'babel-polyfill'

{}

> 斐波那契数的生成器

In [26]:
function* fib(N){
    let n = 0
    let first =0
    let second = 1
    let temp
    while (n < N){
        yield second
        temp= first+second
        first=second
        second = temp
        n +=1     
    } 
    return second
}

'use strict'

In [27]:
for (var i of fib(5)) {
    console.log(i)
}

1
1
2
3
5


undefined

## yield语句


与python中一样,生成器中的yield其实起到一个暂停断点的作用,它的逻辑如下:


+ 遇到yield语句，就暂停执行后面的操作，并将紧跟在yield后面的那个表达式的值，作为返回的对象的value属性值。
+ 下一次调用next方法时，再继续往下执行，直到遇到下一个yield语句。
+ 如果没有再遇到新的yield语句，就一直运行到函数结束，直到return语句为止，并将return语句后面的表达式的值，作为返回的对象的value属性值。
+ 如果该函数没有return语句，则返回的对象的value属性值为undefined。

需要注意的是，yield语句后面的表达式，只有当调用next方法、内部指针指向该语句时才会执行，和python一样,这是js的惰性求值（Lazy Evaluation）的主要实现。

```javascript
function* gen() {
  yield  123 + 456
}
```
上面代码中，yield后面的表达式123 + 456，不会立即求值，只会在next方法将指针移到这一句时，才会求值。

可以从上面的规则看出,js的生成器可以不带yield,这也是和python中不同的地方,
Generator不带yield语句，这时就变成了一个单纯的暂缓执行函数。

In [30]:
function* f() {
  console.log('执行了！')
}

let generator = f()

setTimeout(function () {
  generator.next()
},
2000)

{ _called: false,
  _idleTimeout: 2000,
  _idlePrev: 
   Timer {
     '0': [Function: listOnTimeout],
     _idleNext: [Circular],
     _idlePrev: [Circular],
     msecs: 2000 },
  _idleNext: 
   Timer {
     '0': [Function: listOnTimeout],
     _idleNext: [Circular],
     _idlePrev: [Circular],
     msecs: 2000 },
  _idleStart: 3303829,
  _onTimeout: [Function],
  _repeat: null }

执行了！


上面代码中，函数f如果是普通函数，在为变量generator赋值时就会执行。但是，函数f是一个Generator函数，就变成只有调用next方法时，函数f才会执行。

另外需要注意，yield语句不能用在普通函数中，否则会报错。


下面是另外一个例子。

In [32]:
let arr = [1, [[2, 3], 4], [5, 6]];

let flat = function* (a) {
  let length = a.length;
  for (let i = 0; i < length; i++) {
    let item = a[i]
    if (typeof item !== 'number') {
      yield* flat(item)
    } else {
      yield item
    }
  }
}

for (let f of flat(arr)) {
  console.log(f)
}

1
2
3
4
5
6


undefined

另外，yield语句如果用在一个表达式之中，必须放在圆括号里面。
yield语句用作函数参数或赋值表达式的右边，可以不加括号。

### yield*

如果在Generater函数内部，调用另一个Generator函数，默认情况下是没有效果的。

In [19]:
function* foo() {
  yield 'a'
  yield 'b'
}

function* bar() {
  yield 'x'
  foo()
  yield 'y'
}

for (let v of bar()){
  console.log(v)
}
// "x"
// "y"


x
y


undefined

上面代码中，foo和bar都是Generator函数，在bar里面调用foo，是不会有效果的。

这个就需要用到`yield*`语句，用来在一个Generator函数里面执行另一个Generator函数。


In [33]:
function* bar() {
  yield 'x'
  yield* foo()
  yield 'y'
}
for (let v of bar()){
  console.log(v)
}


x
a
b
y


undefined

`yield*`语句等同于在Generator函数内部，部署一个for...of循环,如果`yield*`后面跟着一个数组，由于数组原生支持遍历器，因此就会遍历数组成员,否则必须使用一个带Iterator接口的对象才行

> 使用`yield*`语句遍历完全二叉树。

In [34]:
// 下面是二叉树的构造函数，
// 三个参数分别是左树、当前节点和右树
function Tree(left, label, right) {
  this.left = left
  this.label = label
  this.right = right
}

// 下面是中序（inorder）遍历函数。
// 由于返回的是一个遍历器，所以要用generator函数。
// 函数体内采用递归算法，所以左树和右树要用yield*遍历
function* inorder(t) {
  if (t) {
    yield* inorder(t.left)
    yield t.label
    yield* inorder(t.right)
  }
}

// 下面生成二叉树
function make(array) {
  // 判断是否为叶节点
  if (array.length == 1) return new Tree(null, array[0], null);
  return new Tree(make(array[0]), array[1], make(array[2]));
}
let tree = make([[['a'], 'b', ['c']], 'd', [['e'], 'f', ['g']]]);

// 遍历二叉树
let result = []
for (let node of inorder(tree)) {
  result.push(node)
}

result

[ 'a', 'b', 'c', 'd', 'e', 'f', 'g' ]

### Generator函数的this

Generator函数总是返回一个遍历器，ES6规定这个遍历器是Generator函数的实例，也继承了Generator函数的prototype对象上的方法。

In [35]:
function* g() {}

g.prototype.hello = function () {
  return 'hi!';
};

let obj = g();

console.log(obj instanceof g) // true
console.log(obj.hello()) // 'hi!'


true
hi!


undefined

上面代码表明，Generator函数g返回的遍历器obj，是g的实例，而且继承了g.prototype。但是，如果把g当作普通的构造函数，并不会生效，因为g返回的总是遍历器对象，而不是this对象。

In [37]:
function* g() {
  this.a = 11
}

let obj = g()
obj.a // undefined

undefined

上面代码中，Generator函数g在this对象上面添加了一个属性a，但是obj对象拿不到这个属性。



In [41]:
function* F(){
  yield this.x = 2
  yield this.y = 3
}

'use strict'


上面代码中，函数F是一个构造函数，又是一个Generator函数。这时，使用new命令就无法生成F的实例了，因为F返回的是一个内部指针。

In [42]:
'next' in (new F())

true

上面代码中，由于new F()返回的是一个Iterator对象，具有next方法，所以上面的表达式为true。


如果要把Generator函数当作正常的构造函数使用，可以采用下面的变通方法。首先，生成一个空对象，使用bind方法绑定Generator函数内部的this。这样，构造函数调用以后，这个空对象就是Generator函数的实例对象了。

In [43]:
function* F(){
  yield this.x = 2
  yield this.y = 3
}
var obj = {}
var f = F.bind(obj)()

console.log(f.next())  // Object {value: 2, done: false}
console.log(f.next()) // Object {value: 3, done: false}
console.log(f.next())  // Object {value: undefined, done: true}

obj // { x: 2, y: 3 }

{ value: 2, done: false }
{ value: 3, done: false }
{ value: undefined, done: true }


{ x: 2, y: 3 }

上面代码中，首先是F内部的this对象绑定obj对象，然后调用它，返回一个Iterator对象。这个对象执行三次next方法（因为F内部有两个yield语句），完成F内部所有代码的运行。这时，所有内部属性都绑定在obj对象上了，因此obj对象也就成了F的实例。

### Generator对象的throw()

Generator函数返回的遍历器对象，都有一个throw方法，可以在函数体外抛出错误，然后在Generator函数体内捕获。

In [44]:
let g = function* () {
  try {
    yield
  } catch (e) {
    console.log('内部捕获', e);
  }
}

let i = g()
i.next()

try {
  i.throw('a')
  i.throw('b')
} catch (e) {
  console.log('外部捕获', e)
}

内部捕获 a
外部捕获 b


undefined

上面代码中，遍历器对象i连续抛出两个错误。第一个错误被Generator函数体内的catch语句捕获。i第二次抛出错误，由于Generator函数内部的catch语句已经执行过了，不会再捕捉到这个错误了，所以这个错误就被抛出了Generator函数体，被函数体外的catch语句捕获。

注意，不要混淆遍历器对象的throw方法和全局的throw命令。上面代码的错误，是用遍历器对象的throw方法抛出的，而不是用throw命令抛出的。后者只能被函数体外的catch语句捕获。

### Generator对象的return()

Generator函数返回的遍历器对象，还有一个return方法，可以返回给定的值，并且终结遍历Generator函数。


In [46]:
function* gen() {
  yield 1;
  yield 2;
  yield 3;
}

var g = gen();

console.log(g.next())        // { value: 1, done: false }
console.log(g.return("foo")) // { value: "foo", done: true }
console.log(g.next())      // { value: undefined, done: true }

{ value: 1, done: false }
{ value: 'foo', done: true }
{ value: undefined, done: true }


undefined

## 作为对象属性的Generator函数

如果一个对象的属性是Generator函数，可以简写成下面的形式

```javascript
let obj = {
  * myGeneratorMethod() {
    ···
  }
}
```

上面代码中，myGeneratorMethod属性前面有一个星号，表示这个属性是一个Generator函数。

完整的写法是:

```javascript
let obj = {
  myGeneratorMethod: function* () {
    // ···
  }
}
```

# 与Iterator接口的关系
任意一个对象的Symbol.iterator方法，等于该对象的遍历器对象生成函数，调用该函数会返回该对象的一个遍历器对象。

遍历器对象本身也有Symbol.iterator方法，执行后返回自身。

In [5]:
function* gen(){
  // some code
}
var g = gen();
g[Symbol.iterator]() === g
// true

true

上面代码中，gen是一个Generator函数，调用它会生成一个遍历器对象g。它的Symbol.iterator属性，也是一个遍历器对象生成函数，执行后返回它自己。

## next方法的参数

yield句本身没有返回值，或者说总是返回undefined。next方法可以带一个参数，该参数就会被当作上一个yield语句的返回值。

In [6]:
function* f() {
  for(var i=0; true; i++) {
    var reset = yield i;
    if(reset) { i = -1; }
  }
}

var g = f();

console.log(g.next()) // { value: 0, done: false }
console.log(g.next()) // { value: 1, done: false }
console.log(g.next(true)) // { value: 0, done: false }

{ value: 0, done: false }
{ value: 1, done: false }
{ value: 0, done: false }


undefined

上面代码先定义了一个可以无限运行的Generator函数f，如果next方法没有参数，每次运行到yield语句，变量reset的值总是undefined。当next方法带一个参数true时，当前的变量reset就被重置为这个参数（即true），因此i会等于-1，下一轮循环就会从-1开始递增。

这个功能有很重要的语法意义。Generator函数从暂停状态到恢复运行，它的上下文状态（context）是不变的。通过next方法的参数，就有办法在Generator函数开始运行之后，继续向函数体内部注入值。也就是说，可以在Generator函数运行的不同阶段，从外部向内部注入不同的值，从而调整函数行为。

In [8]:
function* foo(x) {
  var y = 2 * (yield (x + 1));
  var z = yield (y / 3);
  return (x + y + z);
}

var a = foo(5);
console.log(a.next()) // Object{value:6, done:false}
console.log(a.next()) // Object{value:NaN, done:false}
console.log(a.next()) // Object{value:NaN, done:true}

var b = foo(5);
console.log(b.next()) // { value:6, done:false }
console.log(b.next(12)) // { value:8, done:false }
console.log(b.next(13)) // { value:42, done:true }

{ value: 6, done: false }
{ value: NaN, done: false }
{ value: NaN, done: true }
{ value: 6, done: false }
{ value: 8, done: false }
{ value: 42, done: true }


undefined

上面代码中，第二次运行next方法的时候不带参数，导致y的值等于2 * undefined（即NaN），除以3以后还是NaN，因此返回对象的value属性也等于NaN。第三次运行Next方法的时候不带参数，所以z等于undefined，返回对象的value属性等于5 + NaN + undefined，即NaN。

如果向next方法提供参数，返回结果就完全不一样了。上面代码第一次调用b的next方法时，返回x+1的值6；第二次调用next方法，将上一次yield语句的值设为12，因此y等于24，返回y / 3的值8；第三次调用next方法，将上一次yield语句的值设为13，因此z等于13，这时x等于5，y等于24，所以return语句的值等于42。

**注意，由于next方法的参数表示上一个yield语句的返回值，所以第一次使用next方法时，不能带有参数。V8引擎直接忽略第一次使用next方法时的参数，只有从第二次使用next方法开始，参数才是有效的。从语义上讲，第一个next方法用来启动遍历器对象，所以不用带有参数。**

如果想要第一次调用next方法时，就能够输入值，可以在Generator函数外面再包一层。

In [9]:
function wrapper(generatorFunction) {
  return function (...args) {
    let generatorObject = generatorFunction(...args);
    generatorObject.next();
    return generatorObject;
  };
}

const wrapped = wrapper(function* () {
  console.log(`First input: ${yield}`);
  return 'DONE';
});

wrapped().next('hello!')

First input: hello!


{ value: 'DONE', done: true }

上面代码中，Generator函数如果不用wrapper先包一层，是无法第一次调用next方法，就输入参数的。

In [10]:
function* dataConsumer() {
  console.log('Started');
  console.log(`1. ${yield}`);
  console.log(`2. ${yield}`);
  return 'result';
}

let genObj = dataConsumer();
genObj.next();
// Started
genObj.next('a')
// 1. a
genObj.next('b')
// 2. b

Started
1. a
2. b


{ value: 'result', done: true }

上面代码是一个很直观的例子，每次通过next方法向Generator函数输入值，然后打印出来。